## Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали. <br>
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно. <br>

### Описание данных
Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:<br>
сalls — количество звонков,<br>
minutes — суммарная длительность звонков в минутах,<br>
messages — количество sms-сообщений,<br>
mb_used — израсходованный интернет-трафик в Мб,<br>
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).<br>

## Загрузка и подготовка данных 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [2]:
data = pd.read_csv('users_behavior.csv')

In [3]:
data.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


Данные успешно загружены, обычный датасет, всё в цифрах.

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Нет пропущенных значений, единственное , что можно сделать привести к типу int, количество звонков и сообщений. 

In [5]:
data.duplicated().value_counts()

False    3214
dtype: int64

Дубликатов нет, хорошо. Изменим типы:

In [6]:
data = data.astype({'calls' : 'int', 'messages' : 'int'})

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   int32  
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   int32  
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(2), int32(2), int64(1)
memory usage: 100.6 KB


In [8]:
data.is_ultra.value_counts(normalize = True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

Наблюдаем, что ультрой пользуется лишь 30% от всех данных

Произведем проверку на корреляцию:

In [9]:
data.corr()

,calls,minutes,messages,mb_used,is_ultra
calls,1.000000,0.982083,0.177385,0.286442,0.207122
minutes,0.982083,1.000000,0.173110,0.280967,0.206955
messages,0.177385,0.173110,1.000000,0.195721,0.203830
mb_used,0.286442,0.280967,0.195721,1.000000,0.198568
is_ultra,0.207122,0.206955,0.203830,0.198568,1.000000


In [10]:
data = data.drop(columns = ['calls'])

Отлично, данные подготовлены. 

## Создание и анализ моделей

In [11]:
data_train, data_f, target_train, data_t = train_test_split(data.drop(columns = ['is_ultra']), 
                                                                      data['is_ultra'],
                                                                      train_size = 0.6, 
                                                                      random_state=12345)

In [12]:
data_test, data_valid, target_test, target_valid = train_test_split(data_f, 
                                                                    data_t,
                                                                    train_size = 0.5, 
                                                                    random_state=12345)

In [13]:
print(len(data_train) / len(data))
print(len(data_test) / len(data))
print(len(data_valid) / len(data))

0.5998755444928439
0.2000622277535781
0.2000622277535781


##### Дерево решений
Будем итерировать параметр max_depth. <br>
Отслеживаем точность по метрике accuracy (доля правильных ответов) 

In [14]:
model_2 = 0
model_1 = 0

In [15]:
score_max = 0
act_depth = 0
for depth in range(1, 52, 2):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    model.fit(data_train, target_train)
    predict = model.predict(data_valid)
    score = accuracy_score(target_valid, predict)
    if score > score_max:
        score_max = score
        model_1  = model
        act_depth = depth
score_max, act_depth

(0.7962674961119751, 7)

In [16]:
confusion_matrix(model_1.predict(data_valid), target_valid)

array([[408,  99],
       [ 32, 104]], dtype=int64)

##### Лес решений 
Будем итерировать параметр n_estimators. <br>
Отслеживаем точность по метрике accuracy (доля правильных ответов) 

In [17]:
score_max = 0
act_n_estimators = 0
for n in range(100, 1001, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=n)
    model.fit(data_train, target_train)
    predict = model.predict(data_valid)
    score = accuracy_score(target_valid, predict)
    if score > score_max:
        score_max = score
        model_2  = model
        act_n_estimators = n
score_max, act_n_estimators

(0.7947122861586314, 200)

In [18]:
confusion_matrix(model_2.predict(data_valid), target_valid)

array([[395,  87],
       [ 45, 116]], dtype=int64)

##### Логистическая регрессия 
Отслеживаем точность по метрике accuracy (доля правильных ответов) 

In [19]:
model = LogisticRegression(random_state=12345, max_iter = 3000)
model.fit(data_train, target_train)
predict = model.predict(data_valid)
score = accuracy_score(target_valid, predict)
score

0.6982892690513219

In [20]:
confusion_matrix(model.predict(data_valid), target_valid)

array([[437, 191],
       [  3,  12]], dtype=int64)

Самую высокую точность имеет Лес решений с количеством деревьев 12 - 79,1%  <br>
Но и Дерево решений имеет малую разницу с победителем, при максимальном количество веток 7 - 78,98% <br>
Логистическая регрессия имеет самую малую точность - 70.39%<br>

Т.к. Smart'ом пользуется 70% пользователей из наших данных, поэтому сделаем константную модуль из 0. 

In [21]:
const_model = pd.Series(0, index=target_valid.index) 

In [22]:
accuracy_score(target_valid, const_model)

0.6842923794712286

Ай яй, логистическая регрессия оказалась аналогична константной модели по данной метрике, очень много ошибок 1 рода и она не умеет определять истинно отрицательнные ответы. Другие две модели выйграли в точности, потому что смогли распознать истинно отрицательнные значения.  <br>

In [23]:
print(accuracy_score(model_2.predict(data_test), target_test))
confusion_matrix(model_2.predict(data_test), target_test)

0.7822706065318819


array([[411,  97],
       [ 43,  92]], dtype=int64)

In [24]:
print(accuracy_score(model_1.predict(data_test), target_test))
confusion_matrix(model_1.predict(data_test), target_test)

0.7713841368584758


array([[417, 110],
       [ 37,  79]], dtype=int64)